In [ ]:
%%capture
!pip install plotly

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import requests
import matplotlib.pyplot as plt # matplotlib
import urllib.request # download request
import warnings # to suppress warnings
from datetime import datetime, timedelta
from power_outage_data import load_yearly_data, load_fips_shapes, lat_lon_to_fips
from storm_data import load_tracks, get_timestamps, get_intensity_max
warnings.filterwarnings('ignore')

In [ ]:
yearly_data = load_yearly_data()
outages_2017 = yearly_data[2017]
fips_shapes = load_fips_shapes()

In [ ]:
tks = load_tracks()

# blackout dataset only from 2014
storms = tks.where(tks.season>=2014, drop=True)
storm_ids = []

for i in range(storms.dims['storm']):
    landfall = storms.landfall.values[i]
    sid = storms.sid.values[i]
    if 0 in landfall:
        storm_ids.append(sid)

# filter to landfall storms, NOT currently filtered to US
mask = storms['sid'].isin(storm_ids)
filtered = storms.where(mask, drop=True)

In [ ]:
gdf = gpd.GeoDataFrame.from_features(fips_shapes['features'])
gdf['fips_code'] = gdf['STATE'] + gdf['COUNTY']
gdf = gdf.rename(columns = {'NAME': 'county'})

In [ ]:
def animated_plot(storm):
    example = filtered.sel(storm=20)
    lon_lst, lat_lst = example.lon.values, example.lat.values
    landfall = example.landfall.values
    timestamps, dates, hours = get_timestamps(example)
    intensity = get_intensity(example)

In [ ]:
harvey = pd.DataFrame({'lat': lat_lst, 'lon': lon_lst, 'timestamp': timestamps,
                       'date': dates, 'hour': hours, 'intensity':intensity})
harvey = harvey.dropna()

In [ ]:
# filter to daily max for both outages and storm lat/lon
filtered_outages=outages_2017.drop(columns = 'hour').groupby(by = ['fips_code', 'county', 'state', 'date']).max().reset_index()
harvey_filtered = harvey[['lat', 'lon', 'intensity', 'date']].groupby(by='date').max().reset_index()

# filter outage df to duration of storm
test = filtered_outages.merge(harvey_filtered, on='date', how = 'right').drop(columns=['lat', 'lon'])
test.head()

In [ ]:
# group by state
# county level was too granular for the viz

test_states = test[['state', 'date', 'customers_out', 'intensity']].groupby(['state', 'date']).sum().reset_index()
dates = list(set(test_states.date))
dates = sorted(dates)

In [ ]:
state_dict = {'Alabama':'AL', 'Arkansas': 'AR', 'Connecticut': 'CT',
              'Delaware': 'DE', 'Florida': 'FL', 'Kentucky': 'KY',
              'Louisiana':'LA', 'Maine':'ME', 'Maryland':'MD',
              'Massachusetts':'MA', 'Mississippi':'MS',
              'Missouri':'MO', 'New Hampshire':'NH', 'New Jersey': 'NJ',
              'New York':'NY', 'North Carolina':'NC', 'Oklahoma':'OK',
              'Rhode Island':'RI', 'South Carolina':'SC', 'Tennessee':'TN',
              'Texas': 'TX', 'Virginia':'VA', 'West Virginia':'WV',
              'Georgia':'GA'}

test_states['state_abbr'] = test_states['state'].map(state_dict)

In [ ]:
harvey_filtered = harvey_filtered.sort_values(by = 'date')
latitudes = harvey_filtered.lat
longitudes = harvey_filtered.lon

In [ ]:
scatter_data = []
for i, date in enumerate(dates):
    # Each timestep includes all previous hurricane locations
    for j in range(i + 1):
        scatter_data.append({
            'date': date.strftime('%Y-%m-%d'),
            'lat': latitudes[j],
            'lon': longitudes[j],
            'intensity':float(intensity[j])
        })

df_hurricane = pd.DataFrame(scatter_data)
df_hurricane.date = pd.to_datetime(df_hurricane['date'], format='%Y-%m-%d')
df_hurricane.date = [d.date() for d in df_hurricane.date]
avg_intensity = df_hurricane.intensity.mean() # to scale down intensity point sizes
df_hurricane.intensity = (df_hurricane.intensity / avg_intensity) * 10
df_hurricane.intensity = df_hurricane.intensity.fillna(method='ffill')

# Create base figure
fig = go.Figure()

# Initial Data (First Date)
initial_date = df_hurricane.date[0]
df_initial_hurricane = df_hurricane[df_hurricane['date'] == initial_date]
df_initial_choropleth = test_states[test_states['date'] == initial_date]

# Choropleth layer
fig.add_trace(go.Choropleth(
    locations=df_initial_choropleth['state_abbr'],
    z=df_initial_choropleth['customers_out'],
    locationmode='USA-states',
    colorbar_title='Customers Out of Power',
    name='Choropleth',
    zmin=0, zmax=100000,
    colorscale='Inferno_r'
))

# Hurricane Path (Scattergeo)
fig.add_trace(go.Scattergeo(
    lon=df_initial_hurricane['lon'],
    lat=df_initial_hurricane['lat'],
    mode='markers',
    marker=dict(color='red', symbol='circle', size=df_initial_hurricane['intensity']),
    name='Hurricane Path'
))

# Add Frames for Animation (cumulative hurricane path)
frames = []
for date in df_hurricane['date'].unique():
    df_day_hurricane = df_hurricane[df_hurricane['date'] <= date]  # Include all previous points
    df_day_choropleth = test_states[test_states['date'] == date]
    
    frames.append(go.Frame(
        data=[
            go.Choropleth(locations=df_day_choropleth['state_abbr'],
                          z=df_day_choropleth['customers_out'],
                          zmin=0, zmax=100000),
            go.Scattergeo(lon=df_day_hurricane['lon'], lat=df_day_hurricane['lat'], 
                          mode='markers',
                          marker=dict(color='red', symbol='circle', size = df_day_hurricane['intensity']))
        ],
        name=date.strftime('%Y-%m-%d')
    ))

fig.update(frames=frames)

# Add Slider & Play Button
fig.update_layout(
    updatemenus=[{
        'buttons': [
            {'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'mode': 'immediate'}],
             'label': 'Play',
             'method': 'animate'},
            {'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}],
             'label': 'Pause',
             'method': 'animate'}
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }],
    sliders=[{
        'steps': [{'args': [[date], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}],
                   'label': date.strftime('%Y-%m-%d'), 'method': 'animate'} for date in df_hurricane['date'].unique()],
        'currentvalue': {'prefix': 'Date: ', 'font': {'size': 20}},
        'x': 0.1,
        'y': -0.2
    }]
)

# Layout Settings
fig.update_layout(
    geo=dict(fitbounds='locations'),
    title_text='Hurricane Path with Power Outages',
    height=600, width = 1000
)

fig.show()

In [ ]:
scatter_data = []
for i, date in enumerate(dates):
    # Each timestep includes all previous hurricane locations
    for j in range(i + 1):
        scatter_data.append({
            'date': date.strftime('%Y-%m-%d'),
            'lat': latitudes[j],
            'lon': longitudes[j]
        })

df_hurricane = pd.DataFrame(scatter_data)
df_hurricane.date = pd.to_datetime(df_hurricane['date'], format='%Y-%m-%d')
df_hurricane.date = [d.date() for d in df_hurricane.date]

# Create base figure
fig = go.Figure()

# Initial Data (First Date)
initial_date = df_hurricane.date[0]
df_initial_hurricane = df_hurricane[df_hurricane['date'] == initial_date]
df_initial_choropleth = test[test['date'] == test.date.min()]

# Choropleth layer
fig.add_trace(go.Choropleth(
    locations=df_initial_choropleth['fips_code'],
    z=df_initial_choropleth['customers_out'],
    geojson=fips_shapes,
    colorbar_title='Customers Out of Power',
    name='Choropleth',
    zmin=0, zmax=1000,
    colorscale='Inferno_r'
))

# Hurricane Path (Scattergeo)
fig.add_trace(go.Scattergeo(
    lon=df_initial_hurricane['lon'],
    lat=df_initial_hurricane['lat'],
    mode='markers',
    marker=dict(color='red', symbol='circle'),
    name='Hurricane Path'
))

# Add Frames for Animation (cumulative hurricane path)
frames = []
for date in df_hurricane['date'].unique():
    df_day_hurricane = df_hurricane[df_hurricane['date'] <= date]  # Include all previous points
    df_day_choropleth = test[test['date'] == date]
    
    frames.append(go.Frame(
        data=[
            go.Choropleth(locations=df_day_choropleth['fips_code'],
                          z=df_day_choropleth['customers_out'],
                          zmin=0, zmax=1000),
            go.Scattergeo(lon=df_day_hurricane['lon'], lat=df_day_hurricane['lat'], 
                          mode='markers',
                          marker=dict(color='red', symbol='circle'))
        ],
        name=date.strftime('%Y-%m-%d')
    ))

fig.update(frames=frames)

# Add Slider & Play Button
fig.update_layout(
    updatemenus=[{
        'buttons': [
            {'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'mode': 'immediate'}],
             'label': 'Play',
             'method': 'animate'},
            {'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}],
             'label': 'Pause',
             'method': 'animate'}
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }],
    sliders=[{
        'steps': [{'args': [[date], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}],
                   'label': date.strftime('%Y-%m-%d'), 'method': 'animate'} for date in df_hurricane['date'].unique()],
        'currentvalue': {'prefix': 'Date: ', 'font': {'size': 20}},
        'x': 0.1,
        'y': -0.2
    }]
)

# Layout Settings
fig.update_layout(
    geo=dict(fitbounds='locations'),
    title_text='Hurricane Path with Power Outages',
    height=600, width = 1000
)

fig.show()

In [ ]:
filtered_outages.loc[filtered_outages.fips_code=='01001']

In [ ]:
outages_2017.loc[(outages_2017.fips_code=='01001')]